# Notebook con script para Generar formato para Docentes a partir de la Nomina

### 1. Condiciones iniciales para trabajar con pandas.

In [1]:
import pandas as pd

pd.set_option('display.max_columns', 3000)
pd.set_option('display.max_rows', 3000)

### 2. Carpeta donde esta ubicado la informacion.

In [2]:
cd ~/notebook/David/

[Errno 2] No such file or directory: '/home/bane/notebook/David/'
/home/bane/anaconda3/envs/sist-const-senifa/django_project/Notebook


### 3. Funcion para Ubicar los archivos de cada quincena para un mes en particular.

In [3]:
def Quincenas(carp1, arch1, arch2):
    df1 = pd.read_csv(f'{carp1}{arch1}.csv', sep=';',
                  usecols=['CEDULA','APELLIDO Y NOMBRE', 'Unnamed: 2', 'CARGO', 
                           'UBICACIÓN GEOGRAFICA',  'Unnamed: 9', 'Unnamed: 10', ' ',
                           'NETO A PAGAR'])
    df2 = pd.read_csv(f'{carp1}{arch2}.csv', sep=';',
                      usecols=['CEDULA','APELLIDO Y NOMBRE', 'Unnamed: 2', 'CARGO', 'NETO A PAGAR'])
    return df1, df2

### 4. Funcion para Renombrar y Seleccionar las Columnas de la tabla.
Partimos de la primera quincena para generar la mayoria de la información necesaria.

In [4]:
def Renom_Selecc1(df1):
    df1.rename(columns={"CEDULA":"Cedula", "APELLIDO Y NOMBRE":"Apellido", "Unnamed: 2":"Nombre", 
                       "CARGO":"Cargo", "UBICACIÓN GEOGRAFICA":"Ubicación", "Unnamed: 10":"Año", "Unnamed: 9":"Mes", 
                       " ":"Dia", "NETO A PAGAR":"1era Quincena",}, inplace=True)
    df1 = df1.loc[:, ['Cedula', 'Apellido', 'Nombre', 'Cargo', 'Ubicación', 
                      'Año', 'Mes', 'Dia', '1era Quincena']]        
    return df1

### 5. Funcion para Reorganizar la fecha en la tabla.

In [5]:
def Organiz_Fechas(df1):
    dfA = df1.loc[:, ['Año', 'Mes', 'Dia']]
    dfA['Año'] = dfA['Año'].astype('string')
    dfA['Mes'] = dfA['Mes'].astype('string')
    dfA['Dia'] = dfA['Dia'].astype('string')
    df1['Fecha de Ingreso'] = dfA['Año'] + '-' + dfA['Mes'] + '-' + dfA['Dia']
    df1['Fecha de Ingreso'] = pd.to_datetime(df1['Fecha de Ingreso'], format='%Y/%m/%d')
    del dfA
    df1 = df1.loc[:, ['Cedula', 'Apellido', 'Nombre', 'Cargo', 'Ubicación', 
                      'Fecha de Ingreso', '1era Quincena']]
    df1['1era Quincena'] = df1['1era Quincena'].str.replace(',', '.').astype(float)
    return df1

### 6. Funcion para Renombrar y Seleccionar las columnas de la segunda quincena.

In [6]:
def Renom_Selecc2(df2):
    df2.rename(columns={"CEDULA":"Cedula", "APELLIDO Y NOMBRE":"Apellido", "Unnamed: 2":"Nombre", 
                       "CARGO":"Cargo", "UBICACIÓN GEOGRAFICA":"Ubicación", 
                       "NETO A PAGAR":"2da Quincena",}, inplace=True)
    df2 = df2.loc[:, ['Cedula', 'Apellido', 'Nombre', 'Cargo', '2da Quincena']]
    df2['2da Quincena'] = df2['2da Quincena'].str.replace(',', '.').astype(float)
    return df2

### 7. Funcion para Comprobar transcripcion de datos previo union de ambas tablas.

In [7]:
def Comp_transcrip(df1, df2):
    diff_cedula = df1['Cedula'].equals(df2['Cedula'])
    diff_Apellido = df1['Apellido'].equals(df2['Apellido'])
    diff_Nombre = df1['Nombre'].equals(df2['Nombre'])
    diff_Cargo = df1['Cargo'].equals(df2['Cargo'])
    diferencia = {'diff_cedula':diff_cedula, 'diff_Apellido':diff_Apellido, 
              'diff_Nombre':diff_Nombre, 'diff_Cargo':diff_Cargo}
    for i in range(0,len(diferencia)):
        if list(diferencia.values())[i] == True:
            print(f'Sin problemas en la Columna {df1.columns[i]}')
        else:
            print(f'Se observaron diferencias en las celdas de la Columna {df1.columns[i]}')

### 8. Funcion para Unir tablas y Generar monto total.

In [8]:
def Monto_total(df1, df2):
    df1 = df1.reset_index()
    df2 = df2.reset_index()
    df1 = df1.loc[:, ['Cedula', 'Apellido', 'Nombre', 'Cargo', 'Ubicación', 
                      'Fecha de Ingreso', '1era Quincena']]        
    df1 = df1.set_index('Cedula')
    df2 = df2.loc[:, ['Cedula', '2da Quincena']]
    df2 = df2.set_index('Cedula')
    df1['2da Quincena'] = df2['2da Quincena']
    df1['total'] = df1['1era Quincena'] + df1['2da Quincena']
    del df2
    return df1

### 9. Funcion para separar Madres Comunitarias del resto de la Nomina.

In [9]:
def Separar_madres(df1):
    mask = df1['Cargo'] == 'MADRE COMUNITARIA'
    df1 = df1.loc[mask]
    df2 = df1.loc[mask==False]
    #display(df1.head(), df1.info())
    return df1, df2

In [10]:
def Resultados_Docentes(carp1, arch1, arch2, arch3, arch4):
    tabla1, tabla2 = Quincenas(carp1, arch1, arch2)
    ### Primero creamos las dos tablas con los datos de los archivos con las quincenas.
    tabla1 = Renom_Selecc1(tabla1)         ### Renombranos las columnas tabla 1.
    tabla1 = Organiz_Fechas(tabla1)        ### Reorganizamos la fecha en la tabla 1.
    tabla2 = Renom_Selecc2(tabla2)         ### Renombranos y seleccionamos las columnas tabla 2.
    Comp_transcrip(tabla1, tabla2)         ### Comprobacion de transcripcion antes de unir ambas tablas.
    tabla = Monto_total(tabla1, tabla2)    ### Generacion de tabla final con la información deseada.
    tabla1, tabla2 = Separar_madres(tabla) ### Funcion para separar las madres de la nomina.
    tabla1.to_csv(f'{carp1}{arch3}.csv', sep=';')
    print(f'Generado archivo {arch3}.csv en la carpeta {carp1}')
    tabla2.to_csv(f'{carp1}{arch4}.csv', sep=';')
    print(f'Generado archivo {arch4}.csv en la carpeta {carp1}')
    del tabla, tabla1, tabla2

### 9. Funcion para generacion de archivo con la informacion solicitada.

In [11]:
carp1 = 'Datos/'
arch1 = 'Informacion_RRHH_1era_quincena'
arch2 = 'Informacion_RRHH_2da_quincena'
arch3 = 'Resultados Madres 2020 ENE'
arch4 = 'Resultados Docentes 2020 ENE'
Resultados_Docentes(carp1, arch1, arch2, arch3, arch4)

ValueError: Usecols do not match columns, columns expected but not found: ['Unnamed: 10', ' ', 'Unnamed: 9']

In [ ]:
df1 = pd.read_csv(f'{carp1}{arch1}.csv', sep=';')

In [ ]:
df1.columns

In [ ]:
df1.head()

In [ ]:
df1 = pd.read_csv(f'{carp1}{arch1}.csv', sep=';',
                  usecols=['CEDULA','APELLIDO Y NOMBRE', 'Unnamed: 2', 'CARGO', 
                           'UBICACIÓN GEOGRAFICA',  'Unnamed: 9', 'Unnamed: 10', ' ',
                           'NETO A PAGAR'])

In [ ]:
df1.columns

In [ ]:
df1.head()

In [ ]:
df1.head(50)